# Send a message to Confluent Kafka [Producer]

In [ ]:
!pip install confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 56.9 MB/s eta 0:00:00


In [ ]:
# Required connection configs for kafka producer consumer and admin
from confluent_kafka import Producer
import json
import time
import pandas as pd
import numpy as np

In [ ]:
# create a json
json_records = df.to_dict(orient='records')
json_file = 'first_100_customers.json'
with open(json_file, 'w') as f:
    json.dump(json_records, f, indent=4)
print("File converted to json")

File converted to json


In [ ]:
df = pd.read_csv("/content/first_100_customers.csv")
df.head(5)

,customer_id,name,city,state,country,registration_date,is_active
0,0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False


In [ ]:
# Create a client on confluent
conf = {
    'bootstrap.servers': 'pkc-619z3.us-east1.gcp.confluent.cloud:9092',  # replace with your cluster
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'SZQMKVH3ZUOA7WJX',
    'sasl.password': 'cfltK7z4aPO3ld9Pby9LFlNg607NgoJm5jHri05QUgD92hjFvmVyShyKDqIxi/LA'
}

In [ ]:
producer = Producer(conf)
topic = "ecommerce"

In [ ]:
# Load jason
with open("/content/first_100_customers.json",'r') as file:
  customers_data = json.load(file)

In [ ]:
# Send a single value

value = customers_data[0]
key = value["customer_id"]
print(value)
print(key)

producer.produce(topic, key=key, value=json.dumps(value))
producer.flush()

{'customer_id': 0, 'name': 'Customer_0', 'city': 'Pune', 'state': 'Maharashtra', 'country': 'India', 'registration_date': '2023-06-29', 'is_active': False}
0


TypeError: a bytes-like object is required, not 'int'

In [ ]:
# value and key should be byte like objects. So need to be converted
producer.produce(topic, key= str(key).encode('utf-8'), value=str(value).encode('utf-8'))
producer.flush()

0

In [ ]:
# Callback function that Kafka calls after each message is delivered (or fails).
# This helps us confirm that our messages actually reached the Kafka cluster.
def delivery_report(err, msg):
    if err is not None:
        # If delivery fails, print the error
        print(f'Message delivery failed: {err}')
    else:
        # If delivery succeeds, print topic name and partition where message went
        print(f'Message delivered to {msg.topic()} [Partition: {msg.partition()}]')


# Loop through all customer records we want to send to Kafka
for record in customers_data:
    try:
        # Extract customer_id as the key
        # The key ensures that all messages from the same customer
        # go to the SAME partition (preserves ordering for that customer).
        key = record["customer_id"]

        # Convert the record (dictionary) into a string for sending.
        # Kafka messages must be bytes, so we encode strings into UTF-8.
        value = record
        producer.produce(
            topic,
            key=str(key).encode('utf-8'),       # Convert customer_id into bytes
            value=str(value).encode('utf-8'),   # Convert the whole record into bytes
            callback=delivery_report            # Call delivery_report when send is done
        )

        # Poll Kafka to trigger the delivery_report callback
        # This processes background network events and ensures delivery confirmation.
        producer.poll(1)

    except Exception as e:
        # If any error happens while producing, print it
        print(f"Error: {e}")


# Flush ensures that ALL buffered messages are sent before program exits.
# Without this, some messages may remain in memory and never reach Kafka.
producer.flush()

Message delivered to ecommerce [Partition: 2]
Message delivered to ecommerce [Partition: 2]
Message delivered to ecommerce [Partition: 1]
Message delivered to ecommerce [Partition: 1]
Message delivered to ecommerce [Partition: 1]
Message delivered to ecommerce [Partition: 1]
Message delivered to ecommerce [Partition: 1]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Partition: 2]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Partition: 2]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Partition: 1]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Partition: 2]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Partition: 1]
Message delivered to ecommerce [Partition: 0]
Message delivered to ecommerce [Pa

0

# Recieve data from kafka Cluster [Consumer]

In [ ]:
from confluent_kafka import Consumer, KafkaError
import json
import time

# Required connection configs for kafka consumer
consumer_conf = {
    'bootstrap.servers': 'pkc-619z3.us-east1.gcp.confluent.cloud:9092',  # replace with your cluster
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': 'SZQMKVH3ZUOA7WJX',
    'sasl.password': 'cfltK7z4aPO3ld9Pby9LFlNg607NgoJm5jHri05QUgD92hjFvmVyShyKDqIxi',
    'group.id': 'python-group-1',  # Replace with a unique group ID
    'auto.offset.reset': 'earliest'  # Start consuming from the beginning of the topic
}


In [ ]:
consumer = Consumer(consumer_conf)
consumer

In [ ]:
# Define the Kafka topic to consume from
topic = 'ecommerce'

# Subscribe this consumer to the topic.
# (Consumers can subscribe to multiple topics by passing a list)
consumer.subscribe([topic])


# Define a function to process each message safely
def processes_message(message):
    try:
        # Check if the message has an error
        if message.error():
            # Special case: End of a Kafka partition (not an error, just info)
            if message.error().code() == KafkaError._PARTITION_EOF:
                print('%% %s [%d] reached end at offset %d\n' %
                      (message.topic(), message.partition(), message.offset()))
            # If it's another type of error, raise an exception
            elif message.error():
                raise Exception(message.error())
        else:
            # ✅ Proper message received — print its details
            print('%% %s [%d] at offset %d with key %s:\n' %
                  (message.topic(), message.partition(), message.offset(),
                   str(message.key())))

            # Decode the message value (bytes → string) and print it
            print(json.Load(message.value().decode('utf-8')))

    except Exception as e:
        # Catch any errors in message processing logic
        print(f"Error processing message: {e}")


In [ ]:
# -----------------------
# Poll loop: continuously fetch messages from Kafka
# -----------------------
try:
    print("Listening for messages... (Press Ctrl+C to exit)")

    while True:
        # poll() fetches messages from the broker
        # timeout=1.0 → wait up to 1 second before returning None
        message = consumer.poll(timeout=1.0)

        if(message):
          processes_message(message)

# Allow graceful shutdown with Ctrl+C
except KeyboardInterrupt:
    print("User ended the consumer (KeyboardInterrupt).")

finally:
    # Always close the consumer to free resources and commit offsets safely
    consumer.close()

Listening for messages... (Press Ctrl+C to exit)
User ended the consumer (KeyboardInterrupt).
